In [1]:
!pip install scikit-learn

import copy

from federatedscope.core.configs.config import CN
from federatedscope.register import register_config
from federatedscope.core.configs.yacs_config import Argument

In [2]:
# 1) 기본 cfg 정의
cfg = CN({
    'learning_rate': 0.001,
    'optimizer': CN({'use': False, 'type': 'SGD'}),
    'flag': Argument(
        description='do special',
        required=False,
        default_value=True
    )
})


print(cfg)


# 2) 검사 함수 등록
def prune_opt(cfg):
    if not cfg.optimizer.use and hasattr(cfg.optimizer, 'type'):
        delattr(cfg.optimizer, 'type')




__cfg_check_funcs__: []
__help_info__: {}
flag: True
is_ready_for_run: False
learning_rate: 0.001
optimizer:
  __cfg_check_funcs__: []
  __help_info__: {}
  is_ready_for_run: False
  type: SGD
  use: False


In [3]:
cfg.register_cfg_check_fun(prune_opt)


print(cfg)



__cfg_check_funcs__: [<function prune_opt at 0x7f8cfb9aedc0>]
__help_info__: {}
flag: True
is_ready_for_run: False
learning_rate: 0.001
optimizer:
  __cfg_check_funcs__: []
  __help_info__: {}
  is_ready_for_run: False
  type: SGD
  use: False


In [4]:
# 3) 병합·검사·실행 준비
cfg.merge_from_list(['learning_rate','0.02'])

print(cfg)




__cfg_check_funcs__: [<function prune_opt at 0x7f8cfb9aedc0>]
__help_info__: {'flag': 'do special'}
flag: True
is_ready_for_run: False
learning_rate: 0.02
optimizer:
  __cfg_check_funcs__: []
  __help_info__: {}
  is_ready_for_run: False
  use: False


In [5]:
cfg.clean_unused_sub_cfgs()
print(cfg)
print(type(cfg.flag))


__cfg_check_funcs__: [<function prune_opt at 0x7f8cfb9aedc0>]
__help_info__: {'flag': 'do special'}
flag: True
is_ready_for_run: False
learning_rate: 0.02
optimizer:
  use: False
<class 'federatedscope.core.configs.yacs_config.Argument'>


In [6]:
cfg.de_arguments()
print(cfg)
print(type(cfg.flag))



__cfg_check_funcs__: [<function prune_opt at 0x7f8cfb9aedc0>]
__help_info__: {'flag': 'do special'}
flag: True
is_ready_for_run: False
learning_rate: 0.02
optimizer:
  use: False
<class 'bool'>


In [7]:
# 4) 동결 및 저장
cfg.freeze(save=False)            # outdir/config.yaml 에 최종 설정 기록
print(cfg)

__cfg_check_funcs__: [<function prune_opt at 0x7f8cfb9aedc0>]
__help_info__: {'flag': 'do special'}
flag: True
is_ready_for_run: True
learning_rate: 0.02
optimizer:
  use: False


In [14]:
import copy
from federatedscope.core.configs.config import CN, Argument

# 1) 간단한 설정 딕셔너리로 CN 객체 생성
cfg = CN({
    'alpha': 0.1,
    'beta': 0.2,
    'nested': CN({
        'use': True,
        'param': 123
    }),
    'flag': Argument(description='test flag', required=False, default_value=False)
})

print("=== __dict__ 초기 상태 ===")
# 내부 속성 확인
print({k: v for k, v in cfg.__dict__.items()})







=== __dict__ 초기 상태 ===
{'__immutable__': False, '__deprecated_keys__': set(), '__renamed_keys__': {}, '__new_allowed__': False}


In [15]:
# 잘 보이는 방법 1
print(cfg)  
# → alpha=0.1, beta=0.2, nested.use=True, nested.param=123, flag=Argument(...)



__cfg_check_funcs__: []
__help_info__: {}
alpha: 0.1
beta: 0.2
flag: False
is_ready_for_run: False
nested:
  __cfg_check_funcs__: []
  __help_info__: {}
  is_ready_for_run: False
  param: 123
  use: True


In [19]:
# 잘 보이는 방법 2
cfg.de_arguments()

print(cfg.dump())  

cfg.clear_aux_info()

print(cfg)



__cfg_check_funcs__: []
__help_info__: {}
alpha: 0.1
beta: 0.2
flag: false
is_ready_for_run: false
nested:
  __cfg_check_funcs__: []
  __help_info__: {}
  is_ready_for_run: false
  param: 123
  use: true

alpha: 0.1
beta: 0.2
flag: False
nested:
  param: 123
  use: True


In [ ]:
# 2) __getattr__ 테스트: 기존 키와 존재하지 않는 키
print("\nalpha via dot:", cfg.alpha)
try:
    print("gamma via dot:", cfg.gamma)
except AttributeError as e:
    print("존재하지 않는 속성 접근 시:", e)

# 3) __delattr__ 테스트: beta 삭제
print("\n=== 'beta' 삭제 전 keys ===", list(cfg.keys()))
del cfg.beta
print("=== 'beta' 삭제 후 keys === ", list(cfg.keys()))




alpha via dot: 0.1
존재하지 않는 속성 접근 시: gamma

=== 'beta' 삭제 전 keys === ['alpha', 'beta', 'nested', 'flag']
=== 'beta' 삭제 후 keys ===  ['alpha', 'nested', 'flag']

=== clear_aux_info 전 __dict__ 요소 ===
{}

=== clear_aux_info 후 __dict__ 요소 ===
{}


In [11]:
import copy
import os
from pathlib import Path

from federatedscope.core.configs.config import CN, Argument
import logging
logging.basicConfig(level=logging.WARNING)

def set_help_info(cn_node, help_info_dict, prefix=""):
    for k, v in cn_node.items():
        if isinstance(v, Argument) and k not in help_info_dict:
            help_info_dict[prefix + k] = v.description
        elif isinstance(v, CN):
            set_help_info(v,
                          help_info_dict,
                          prefix=f"{k}." if prefix == "" else f"{prefix}{k}.")

# ────────────────────────────────────────────────────────────────────────
# 1) 베이스 CN 객체 정의
# ────────────────────────────────────────────────────────────────────────
base_cfg = CN({
    'learning_rate': 0.001,
    'batch_size': 32,
    'optimizer': CN({
        'use': False,
        'type': 'SGD',
        'momentum': 0.9
    }),
    'special_flag': Argument(
        description='run in special mode',
        required=False,
        default_value=True
    ),
    # outdir/wandb 같은 일반적인 필드도 미리 추가해 줍니다
    'outdir': './exp_test',
    'wandb': CN({
        'use': False,
        'project': 'demo'
    })
})

# ────────────────────────────────────────────────────────────────────────
# 2) 검사 함수 정의 및 등록
# ────────────────────────────────────────────────────────────────────────
def prune_optimizer(cfg):
    if not cfg.optimizer.use:
        for k in list(cfg.optimizer.keys()):
            if k != 'use':
                delattr(cfg.optimizer, k)

def clamp_lr(cfg):
    if cfg.learning_rate < 0.01:
        cfg.learning_rate = 0.01

base_cfg.register_cfg_check_fun(prune_optimizer)
base_cfg.register_cfg_check_fun(clamp_lr)

# ────────────────────────────────────────────────────────────────────────
# 3) 도움말 출력 (print_help)
# ────────────────────────────────────────────────────────────────────────
print("=== 전체 옵션 도움말 ===")
base_cfg.print_help()

print("\n=== learning_rate 도움말 ===")
base_cfg.print_help('learning_rate')

print("\n=== learning_rate 도움말 ===")
# (2) set_help_info 로 __help_info__ 재생성
set_help_info(base_cfg, base_cfg.__help_info__)

# (3) 이제 도움말 출력
base_cfg.print_help()

# ────────────────────────────────────────────────────────────────────────
# 4) 필수 인자 체크 (check_required_args)
# ────────────────────────────────────────────────────────────────────────
# 예시로 required=True인 항목을 하나 추가해 보겠습니다
base_cfg['must_set'] = Argument(
    description='you must set this manually',
    required=True,
    default_value=None
)
print("\n=== check_required_args 실행 (warning 발생 예상) ===")
base_cfg.check_required_args()

# ────────────────────────────────────────────────────────────────────────
# 5) ready_for_run(): assert_cfg → clean_unused_sub_cfgs → check_required_args → de_arguments
# ────────────────────────────────────────────────────────────────────────
print("\n=== ready_for_run 실행 전 ===")
print(base_cfg)

base_cfg.ready_for_run()
print("\n=== ready_for_run 실행 후 ===")
print(base_cfg)
#  - optimizer.use=False → optimizer.type/momentum 삭제
#  - special_flag: Argument → True
#  - must_set(required) 경고 (value=None)

# ────────────────────────────────────────────────────────────────────────
# 6) freeze(): ready_for_run 후 불변화, config.yaml 저장, wandb 업데이트
# ────────────────────────────────────────────────────────────────────────
# (outdir 디렉토리를 지워 두었다가 실행해 보세요)
if os.path.isdir(base_cfg.outdir):
    print(f"** 경고: '{base_cfg.outdir}' 디렉토리가 이미 존재합니다. 삭제 후 다시 시도하세요.")
else:
    base_cfg.freeze(save=True, inform=True)
    print(f"config.yaml 이 {base_cfg.outdir}/config.yaml 로 저장되었습니다.")


2025-06-12 03:13:12,575 (config:182)WARNING: You have not set the required argument must_set
2025-06-12 03:13:12,577 (config:182)WARNING: You have not set the required argument must_set


=== 전체 옵션 도움말 ===

=== learning_rate 도움말 ===

=== learning_rate 도움말 ===
  --special_flag 	 run in special mode

=== check_required_args 실행 (warning 발생 예상) ===

=== ready_for_run 실행 전 ===
__cfg_check_funcs__: [<function prune_optimizer at 0x7ff1b01f49d0>, <function clamp_lr at 0x7fefa5302ca0>]
__help_info__: {'special_flag': 'run in special mode'}
batch_size: 32
is_ready_for_run: False
learning_rate: 0.001
must_set: Required(NoneType)
optimizer:
  __cfg_check_funcs__: []
  __help_info__: {}
  is_ready_for_run: False
  momentum: 0.9
  type: SGD
  use: False
outdir: ./exp_test
special_flag: True
wandb:
  __cfg_check_funcs__: []
  __help_info__: {}
  is_ready_for_run: False
  project: demo
  use: False

=== ready_for_run 실행 후 ===
__cfg_check_funcs__: [<function prune_optimizer at 0x7ff1b01f49d0>, <function clamp_lr at 0x7fefa5302ca0>]
__help_info__: {'special_flag': 'run in special mode'}
batch_size: 32
is_ready_for_run: True
learning_rate: 0.01
must_set: None
optimizer:
  use: False
outdi